# Station isochrone Generation

This notebook aims to try and generate isochrones of train stations

It will first attempt to merge isochrones of station entrances where available

Otherwise it will be a simple isochrone generation of the approximate centre of the station



In [50]:
#setting up supabase client
import os
import pandas as pd
from supabase import create_client, Client

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import shape
from geopandas import GeoSeries
from shapely.geometry import Polygon


#in the py webapp, keys will be stored in github secrets
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(url, key)

In [2]:
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import os

import sys

# Get the path of the parent directory (the root of the project)
module_path = os.path.abspath(os.path.join('..'))

# Add the parent directory to sys.path
sys.path.append(module_path)

# Now you can import key.py, this is only done for testing purposes. actual live version should be stored in secret github
from key import ORS_KEY

#Personal api_key stored in a key.py file that is in gitignore. Uncomment the following and provide your own ORS api for your own use
## api_key = "you api key " #Provide your personal API key
ors_key: str = ORS_KEY


ors_client = client.Client(key=ors_key) 

# Querying station entrances from Supabase


The following blocks will load the various relevant tables from Supabase

In [3]:
#Querying table with entrance isochrones

#query to supabase using the previous supabase client that was declared
response_entrances_isochrones = supabase.table('entrances_isochrones').select("*").execute()

data,_ = response_entrances_isochrones
entrances_isochrones_df = pd.DataFrame(data[1])

# Convert the geometry column from dictionaries to shapely Polygons
entrances_isochrones_df['geometry'] = entrances_isochrones_df['geometry'].apply(lambda geom: shape(geom))

# Convert the DataFrame to a GeoDataFrame
entrances_isochrones_gdf = gpd.GeoDataFrame(entrances_isochrones_df, geometry='geometry', crs='EPSG:4326')


entrances_isochrones_gdf

,isochrone_id,value,center,area,reachfactor,total_pop,type,entrance_id,geometry
0,11187840754900,900,"[101.6956212881515, 3.178221134305872]",2562043.68,0.8155,21957,Polygon,11187840754,"POLYGON ((101.68708 3.17785, 101.68752 3.17655..."
1,3308608989300,300,"[101.7125217751169, 3.1588555818871913]",337860.68,0.9679,4774,Polygon,3308608989,"POLYGON ((101.70906 3.15892, 101.70922 3.15865..."
2,7259253706300,300,"[101.6991072, 3.1590919]",285653.74,0.8183,2842,Polygon,7259253706,"POLYGON ((101.69596 3.15864, 101.69716 3.15671..."
3,3308608989600,600,"[101.7125217751169, 3.1588555818871913]",1362846.78,0.9761,18751,Polygon,3308608989,"POLYGON ((101.70580 3.15729, 101.70579 3.15715..."
4,4092013971300,300,"[101.6141298, 3.0222312]",220091.61,0.6305,1498,Polygon,4092013971,"POLYGON ((101.61146 3.02337, 101.61197 3.02128..."
...,...,...,...,...,...,...,...,...,...
877,11039579994600,600,"[101.69373096126859, 3.1670089963140877]",1251497.64,0.8963,15656,Polygon,11039579994,"POLYGON ((101.68883 3.16710, 101.68884 3.16370..."
878,11039579994900,900,"[101.69373096126859, 3.1670089963140877]",2652413.84,0.8443,30134,Polygon,11039579994,"POLYGON ((101.68610 3.16642, 101.68672 3.16494..."
879,11061429685600,600,"[101.6720750549697, 3.2146979016680297]",1084752.26,0.7769,6835,Polygon,11061429685,"POLYGON ((101.66858 3.21536, 101.66857 3.21528..."
880,11061429685900,900,"[101.6720750549697, 3.2146979016680297]",1969864.91,0.6270,12886,Polygon,11061429685,"POLYGON ((101.66857 3.21528, 101.66693 3.21038..."


In [4]:
#query to supabase using the previous supabase client that was declared
response_stations = supabase.table('stations').select("*").eq('region','Klang Valley').execute()

data,_ = response_stations

stations_df = pd.DataFrame(data[1])
# Create a new column in your DataFrame for the geographic data
stations_df['geometry'] = [Point(xy) for xy in zip(stations_df['longitude'], stations_df['latitude'])]

# Convert the DataFrame to a GeoDataFrame
stations_gdf = gpd.GeoDataFrame(stations_df, geometry='geometry')
# Set the coordinate reference system (CRS) to EPSG:4326 (WGS84)
stations_gdf.crs = "EPSG:4326"

stations_gdf

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened,station_id,geometry
0,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,1,POINT (101.69379 3.13951)
1,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,2,POINT (101.69301 3.15454)
2,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,3,POINT (101.69123 3.16500)
3,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,4,POINT (101.67899 3.11853)
4,Seputeh,KB02,Keretapi Tanah Melayu,3.113697,101.681299,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,5,POINT (101.68130 3.11370)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Sri Andalas,JS22,Rapid KL,3.015225,101.440441,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,255,POINT (101.44044 3.01522)
255,Klang Jaya,JS23,Rapid KL,3.005072,101.442081,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,256,POINT (101.44208 3.00507)
256,Bandar Bukit Tinggi,JS24,Rapid KL,2.993526,101.446175,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,257,POINT (101.44617 2.99353)
257,Johan Setia,JS26,Rapid KL,2.975436,101.460718,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,258,POINT (101.46072 2.97544)


In [5]:
#query to supabase using the previous supabase client that was declared
response_stations = supabase.table('station_entrances').select("*").execute()

data,_ = response_stations

stations_entrances_df = pd.DataFrame(data[1])
stations_entrances_df

,relationship_id,entrance_id,station_name,station_code
0,0,10796851698,Pudu,AG10
1,1,10796851698,Pudu,SP10
2,2,5485710279,Kampung Baru,KJ11
3,3,5485710278,Kampung Baru,KJ11
4,4,9740843587,Masjid Jamek (KJ),KJ13
...,...,...,...,...
324,324,5044809585,Tun Razak Exchange (PY),PY23
325,325,5044809586,Tun Razak Exchange (PY),PY23
326,326,12155642296,Kajang 2,KB06A
327,327,12155642293,Kajang 2,KB06A


## Task TODO

1. Left join station_entrances with entrances_isochrones_gdf on entrance_id
2. This will create a table with isochrones mapped to station_name
3. Iterate stations table to look up joined table for entrance isochrones.
4. If isochrones are found, store all of them and use a unity function to combine them all
5. save in a new isochrones polygon column just for stations
6. If no isochrones found, just query on station coordinates

In [6]:
#correcting the column data type to prep for merging
entrances_isochrones_gdf['entrance_id']= entrances_isochrones_gdf['entrance_id'].astype(int)  

In [7]:
# Step 1: Left join station_entrances with entrances_isochrones_gdf on entrance_id
# This will create a table where each station entrance has corresponding isochrones mapped
merged_gdf = stations_entrances_df.merge(entrances_isochrones_gdf, on='entrance_id', how='left')
merged_gdf

,relationship_id,entrance_id,station_name,station_code,isochrone_id,value,center,area,reachfactor,total_pop,type,geometry
0,0,10796851698,Pudu,AG10,10796851698300,300.0,"[101.7121408, 3.1347568]",278905.63,0.7990,2478.0,Polygon,"POLYGON ((101.70965 3.13579, 101.70971 3.13562..."
1,0,10796851698,Pudu,AG10,10796851698600,600.0,"[101.7121408, 3.1347568]",1366234.08,0.9785,13013.0,Polygon,"POLYGON ((101.70670 3.13675, 101.70656 3.13563..."
2,0,10796851698,Pudu,AG10,10796851698900,900.0,"[101.7121408, 3.1347568]",2968558.89,0.9449,26537.0,Polygon,"POLYGON ((101.70388 3.13690, 101.70406 3.13641..."
3,1,10796851698,Pudu,SP10,10796851698300,300.0,"[101.7121408, 3.1347568]",278905.63,0.7990,2478.0,Polygon,"POLYGON ((101.70965 3.13579, 101.70971 3.13562..."
4,1,10796851698,Pudu,SP10,10796851698600,600.0,"[101.7121408, 3.1347568]",1366234.08,0.9785,13013.0,Polygon,"POLYGON ((101.70670 3.13675, 101.70656 3.13563..."
...,...,...,...,...,...,...,...,...,...,...,...,...
980,327,12155642293,Kajang 2,KB06A,12155642293600,600.0,"[101.7924757365604, 2.962197483109656]",1132110.23,0.8108,5671.0,Polygon,"POLYGON ((101.78562 2.96171, 101.78582 2.96056..."
981,327,12155642293,Kajang 2,KB06A,12155642293900,900.0,"[101.7924757365604, 2.962197483109656]",2647963.56,0.8429,13241.0,Polygon,"POLYGON ((101.78315 2.96410, 101.78342 2.96268..."
982,328,12155642294,Kajang 2,KB06A,12155642294300,300.0,"[101.7920585756707, 2.962773292823094]",275078.71,0.7880,1118.0,Polygon,"POLYGON ((101.78944 2.96378, 101.78966 2.96172..."
983,328,12155642294,Kajang 2,KB06A,12155642294600,600.0,"[101.7920585756707, 2.962773292823094]",1279931.08,0.9167,5476.0,Polygon,"POLYGON ((101.78577 2.96369, 101.78622 2.96092..."


In [133]:
#Iterate stations_gdf table to look up merged_gdf table for entrance isochrones.
#If isochrones are found in merged_gdf, 

from shapely.ops import unary_union
from geopandas import GeoDataFrame


def generate_station_isochrone(client,station_id,reach_centre,range=900,interval=300):
    isochrone_parameters = {
              'profile': 'foot-walking', 
              'range': [range], # 900/60 = 15 mins
              'interval': interval,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }

    #reach_centre is in the format: [101.7127175, 3.1587619], ie [longitude,latitude]
    isochrone_parameters['locations'] = [reach_centre]
    isochrones_output = client.isochrones(**isochrone_parameters)

    df = pd.json_normalize(isochrones_output['features'])
    df = df.drop(columns=['type','properties.group_index'])

    #reformate coordinates column
    df['geometry.coordinates']=df['geometry.coordinates'].apply(lambda row:row[0])

    # Remove 'properties.' and 'geometry.' from the column names
    df.columns = df.columns.str.replace('properties.', '', regex=False)
    df.columns = df.columns.str.replace('geometry.', '', regex=False)
    

    #creating a new id key for isochrones. to be inserted in supabase table
    df['station_id']=station_id
    df['isochrone_id'] = df['station_id'].astype(str) + df['value'].astype(int).astype(str)


    # Convert list of coordinates to Polygon objects
    df['geometry'] = df['coordinates'].apply(Polygon)
    df.drop(columns=['coordinates'],inplace=True)
    
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
    #gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)
    gdf['area'] = gdf['area'].astype(float)
    gdf['reachfactor'] = gdf['reachfactor'].astype(float)
    gdf['total_pop'] = gdf['total_pop'].astype(float)
    #area calculation likely wrong. just trying to standardize method
    gdf['area'] = gdf['geometry'].area

    return gdf   

def get_station_isochrone(station_row, merged_gdf,ors_client):
    station_code = station_row['station_code']
    
    # Find all entrances for this station
    station_entrances = merged_gdf[merged_gdf['station_code'] == station_code]
    
    if not station_entrances.empty:
        print("combining isochrones")
        
        average_pop = station_entrances.groupby('value')['total_pop'].mean()
        average_reach_factor = station_entrances.groupby('value')['reachfactor'].mean()

        reach_centre = station_entrances['center'].iloc[0]

        # Group the geometries by 'value' and create unions
        grouped_isochrones = station_entrances.groupby('value')['geometry'].apply(unary_union)
        # Create a new DataFrame with the combined isochrones
        combined_isochrones = gpd.GeoDataFrame(
            {'value': grouped_isochrones.index,
             'center':reach_centre,
             'reachfactor':average_reach_factor,
             'total_pop':average_pop},
            geometry=grouped_isochrones.values,
            crs="EPSG:4326"
        )
        #creating a new id key for isochrones. to be inserted in supabase table
        combined_isochrones['station_id']=station_row['station_id']
        combined_isochrones['isochrone_id'] = combined_isochrones['station_id'].astype(str) + combined_isochrones['value'].astype(int).astype(str)        
        #area calculation likely wrong. just trying to standardize method
        combined_isochrones['area'] = combined_isochrones['geometry'].area
        combined_isochrones['type'] = combined_isochrones['geometry'].type
        combined_isochrones = combined_isochrones.reset_index(drop=True)
        
        # If you need to keep other columns from station_row, you can merge them here
        # For example:
        # result = pd.concat([station_row.drop('geometry', axis=1), combined_isochrones], axis=1)
        
        return combined_isochrones
    
    else:
        print("generating isochrones")
        # If no entrances found, generate new isochrone based on station coordinates
        # Extract coordinates (reach_centre) from the geometry
        reach_centre = [station_row['geometry'].coords[0][0], station_row['geometry'].coords[0][1]]

        return generate_station_isochrone(ors_client,station_row['station_id'],reach_centre)

    

def update_station_isochrone_supabase(supabase_client,isochrone_geodataframe):
    # # Convert list of coordinates to Polygon objects
    # isochrone_dataframe['geometry'] = isochrone_dataframe['coordinates'].apply(Polygon)
    # isochrone_dataframe.drop(columns=['coordinates'],inplace=True)
    
    # # Convert to GeoDataFrame
    # isochrone_geodataframe = gpd.GeoDataFrame(isochrone_dataframe, geometry='geometry', crs="EPSG:4326")
    isochrone_geodataframe['geometry'] = isochrone_geodataframe['geometry'].apply(lambda geom: geom.wkt)
    # isochrone_geodataframe['area'] = isochrone_geodataframe['area'].astype(float)
    # isochrone_geodataframe['reachfactor'] = isochrone_geodataframe['reachfactor'].astype(float)
    # isochrone_geodataframe['total_pop'] = isochrone_geodataframe['total_pop'].astype(float)

    isochrone_dict = isochrone_geodataframe.to_dict(('records'))

    #upsert will insert the row if it doesn’t exist or update the row if it already exists. 
    data, error = supabase_client.table('station_isochrones').upsert(isochrone_dict).execute()

    return data,error

In [114]:
stations_gdf.iloc[1]

name                                       Bank Negara
station_code                                      KA03
service_provider_name            Keretapi Tanah Melayu
latitude                                      3.154542
longitude                                    101.69301
route_id                                            KA
route_name                               Seremban Line
line_number                                          1
line_colour                                       Blue
colour_hex_code                                #0000FF
region                                    Klang Valley
odonym                                            None
namesake                                          None
opened                                            None
station_id                                           2
geometry                 POINT (101.6930105 3.1545422)
Name: 1, dtype: object

In [115]:
sample_combined = get_station_isochrone(stations_gdf.iloc[8],merged_gdf)

combining isochrones


/var/folders/2s/3b55fy1174l4bfchw66wdl800000gn/T/ipykernel_92403/1834069666.py:80: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  combined_isochrones['area'] = combined_isochrones['geometry'].area


In [118]:
sample_generated = get_station_isochrone(stations_gdf.iloc[9],merged_gdf)

generating isochrones


/var/folders/2s/3b55fy1174l4bfchw66wdl800000gn/T/ipykernel_92403/3341404757.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf['geometry'].area


In [119]:
sample_generated

,value,center,area,reachfactor,total_pop,type,station_id,isochrone_id,geometry
0,300.0,"[101.78770801629291, 2.9395100221711816]",0.000020,0.7131,839.0,Polygon,10,10300,"POLYGON ((101.78521 2.94117, 101.78507 2.93850..."
1,600.0,"[101.78770801629291, 2.9395100221711816]",0.000064,0.5661,3355.0,Polygon,10,10600,"POLYGON ((101.78476 2.94308, 101.78481 2.94041..."
2,900.0,"[101.78770801629291, 2.9395100221711816]",0.000168,0.6630,7410.0,Polygon,10,10900,"POLYGON ((101.78450 2.94634, 101.78459 2.94517..."


In [120]:
sample_combined

,value,center,reachfactor,total_pop,geometry,station_id,isochrone_id,area,type
0,300.0,"[101.79089, 2.9838212]",0.57035,1331.5,"POLYGON ((101.79052 2.98026, 101.79001 2.97965...",9,9300,0.000021,Polygon
1,600.0,"[101.79089, 2.9838212]",0.69160,7168.5,"POLYGON ((101.79182 2.97781, 101.79003 2.97581...",9,9600,0.000090,Polygon
2,900.0,"[101.79089, 2.9838212]",0.71805,19265.0,"POLYGON ((101.78663 2.98590, 101.78657 2.98624...",9,9900,0.000199,Polygon


In [136]:
# Apply the function to each row in stations_gdf and update in Supabase

def process_stations(stations_gdf, merged_gdf,ors_client, supabase_client):
    # Iterate over each row in the GeoDataFrame
    for index, row in stations_gdf.iterrows():

        try:
            isochrone_dataframe = get_station_isochrone(
                row, 
                merged_gdf,
                ors_client)
            
            # Update the isochrone data in Supabase using the specified function
            update_station_isochrone_supabase(supabase_client, isochrone_dataframe)
            print(f'updated {row.station_id}')
            
        except Exception as e:
            # Handle exceptions (e.g., API errors, invalid geometries, etc.)
            print(f"Error processing entrance {row.station_id}: {e}")

In [137]:
process_stations(stations_gdf,merged_gdf,ors_client,supabase)

generating isochrones


/var/folders/2s/3b55fy1174l4bfchw66wdl800000gn/T/ipykernel_92403/2605401413.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf['geometry'].area
/Users/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


updated 1
generating isochrones


/var/folders/2s/3b55fy1174l4bfchw66wdl800000gn/T/ipykernel_92403/2605401413.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf['geometry'].area
/Users/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


updated 2
generating isochrones


...

/var/folders/2s/3b55fy1174l4bfchw66wdl800000gn/T/ipykernel_92403/2605401413.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf['geometry'].area
/Users/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


updated 257
generating isochrones


/var/folders/2s/3b55fy1174l4bfchw66wdl800000gn/T/ipykernel_92403/2605401413.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf['geometry'].area
/Users/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


updated 258
generating isochrones
updated 0


/var/folders/2s/3b55fy1174l4bfchw66wdl800000gn/T/ipykernel_92403/2605401413.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['area'] = gdf['geometry'].area
/Users/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [138]:
#testing to display some isochrones

#Querying table with entrance isochrones
from shapely.geometry import shape

#query to supabase using the previous supabase client that was declared
response_station_isochrones = supabase.table('station_isochrones').select("*").execute()

data,_ = response_station_isochrones
station_isochrones_df = pd.DataFrame(data[1])

# Convert the geometry column from dictionaries to shapely Polygons
station_isochrones_df['geometry'] = station_isochrones_df['geometry'].apply(lambda geom: shape(geom))

# Convert the DataFrame to a GeoDataFrame
entrances_isochrones_gdf = gpd.GeoDataFrame(station_isochrones_df, geometry='geometry', crs='EPSG:4326')


import folium
import geopandas as gpd

# Define the center of the map based on the GeoDataFrame's centroid
map_center = entrances_isochrones_gdf.geometry.centroid.iloc[0].coords[0][::-1]

# Create a folium map centered on the geometries
m = folium.Map(location=map_center, zoom_start=12)

# Add GeoDataFrame geometries to the map
folium.GeoJson(
    entrances_isochrones_gdf[entrances_isochrones_gdf['value']==300],
    name="Entrances Isochrones",
    style_function=lambda feature: {
        'fillColor': 'cyan',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    }
).add_to(m)

# Add a layer control panel
folium.LayerControl().add_to(m)

# Display the map
m.save("entrances_isochrones_map.html")
m

/var/folders/2s/3b55fy1174l4bfchw66wdl800000gn/T/ipykernel_92403/1021231826.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_center = entrances_isochrones_gdf.geometry.centroid.iloc[0].coords[0][::-1]
